Inisiasi Program

In [1]:
from pf_simulation import PowerFactorySim as pfsim
import os
import pandas as pd

FOLDER_NAME = ''
PROJECT_NAME = '39 Bus New England System'
STUDY_CASE_NAME = '1. Power Flow'
VARIABLES = {
    'name': {
        '*.ElmTerm': ['magnitude','angle'],
        '*.ElmSym': ['rotor_angle', 'out_of_step']
    },
    'symbol': {
        '*.ElmTerm': ['m:u1', 'm:phiu'],
        '*.ElmSym': ['c:firel', 's:outofstep']
    }
}

sim = pfsim(folder_name=FOLDER_NAME, project_name=PROJECT_NAME, study_case_name=STUDY_CASE_NAME)
# sim.app.Show()

# Membuat skenario operasi
SCENARIO = {
    'f_line' : sim.app.GetCalcRelevantObjects('*.ElmLne'),
    'f_location' : [0, 20, 50, 80],
    'f_duration' : [0.1, 0.15, 0.18, 0.2, 0.25],
    'load_level' : [i/100 for i in range(50,115,5)],
}
scenario_metadata = sim.createScenario(scenario_dict=SCENARIO, save=True)

# Memuat nilai default load dan gen
file_dir = "output/default_data"
load_file = f"{file_dir}/load_data.parquet"
gen_file = f"{file_dir}/generator_data.parquet"
if all(map(os.path.exists, [load_file, gen_file])):
    load_default, gen_default = (pd.read_parquet(load_file), pd.read_parquet(gen_file))
else:
    load_default, gen_default = sim.loadGenData()
    load_default.to_parquet(load_file, index=False)
    gen_default.to_parquet(gen_file, index=False)

# Mengatur parameter untuk simulasi optimal power flow
sim.opfSetup()

# Mengatur parameter beban ke nilai default
sim.loadSetup(1, load_default)
sim.opfAnalysis('los')
sim.ldfAnalysis()

display(scenario_metadata)

,scenario,load_level,f_line,f_location,f_duration
0,scenario_0001,0.5,<l3>\Fathin.IntUser\39 Bus New England System....,0,0.10
1,scenario_0002,0.5,<l3>\Fathin.IntUser\39 Bus New England System....,0,0.15
2,scenario_0003,0.5,<l3>\Fathin.IntUser\39 Bus New England System....,0,0.18
3,scenario_0004,0.5,<l3>\Fathin.IntUser\39 Bus New England System....,0,0.20
4,scenario_0005,0.5,<l3>\Fathin.IntUser\39 Bus New England System....,0,0.25
...,...,...,...,...,...
8835,scenario_8836,1.1,<l3>\Fathin.IntUser\39 Bus New England System....,80,0.10
8836,scenario_8837,1.1,<l3>\Fathin.IntUser\39 Bus New England System....,80,0.15
8837,scenario_8838,1.1,<l3>\Fathin.IntUser\39 Bus New England System....,80,0.18
8838,scenario_8839,1.1,<l3>\Fathin.IntUser\39 Bus New England System....,80,0.20


Testing OPF

In [5]:
file_dir = "output/generator_dispatch"
for load_lvl_iter in sim.load_level:
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        print(f'{load_lvl_iter:.2f} load, OPF min los failed, trying min dev')
        sim.opfAnalysis('dev')
    
    # Lakukan analisis daya dan ambil dispatch
    gen_dispatch = sim.ldfAnalysis().gen

    # Simpan hasil ke HDF5
    gen_dispatch.to_parquet(f'{file_dir}/scenario_{int(load_lvl_iter*100)}.parquet', index=False)
    print(f'{load_lvl_iter:.2f} load, OPF and LF running successfully')

0.50 load, OPF and LF running successfully
0.55 load, OPF and LF running successfully
0.60 load, OPF and LF running successfully
0.65 load, OPF and LF running successfully
0.70 load, OPF and LF running successfully
0.75 load, OPF min los failed, trying min dev
0.75 load, OPF and LF running successfully
0.80 load, OPF and LF running successfully
0.85 load, OPF and LF running successfully
0.90 load, OPF and LF running successfully
0.95 load, OPF and LF running successfully
1.00 load, OPF and LF running successfully
1.05 load, OPF and LF running successfully
1.10 load, OPF and LF running successfully


In [5]:
sc

# f_loc_iter = f_location[0]
# f_line_iter = f_line[0]
# f_dur_iter = f_duration[0]
# load_lvl_iter = 1

# sim.scSetup(f_loc_iter, f_line_iter, f_dur_iter)
# sim.rmsSimulation(VARIABLES['symbol'], t_start=-100, t_step=10, t_stop=30)
# base_dir = 'D:\Fathin\github\powerfactory_rms\output\dataset'
# sim.getRmsResult(base_dir scenario)


In [ ]:
def job():
    sim.loadSetup(load_lvl_iter, load_default)
    try:
        sim.opfAnalysis('los')
    except AttributeError:
        sim.opfAnalysis('dev')
    sim.scSetup(f_loc_iter, f_line_iter, f_dur_iter)
    sim.rmsSimulation(VARIABLES['symbol'], t_start=-100, t_step=10, t_stop=30)
    
    

In [55]:
from tqdm import tqdm
import time
import pandas as pd

restart = True

# Baca file metadata
scenario_list = pd.read_csv("output/scenario_metadata.csv")

# Pastikan kolom 'iter_time' ada
if 'iter_time' not in scenario_list.columns:
    scenario_list['iter_time'] = None

# Filter skenario yang belum dijalankan
if restart:
    scenario_list['iter_time'] = None  # Reset semua iterasi
scenario_list_run = scenario_list[scenario_list['iter_time'].isna()].reset_index(drop=True)

# Total iterasi yang tersisa
total_iterations = scenario_list_run.shape[0]

# Mulai simulasi
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    for _, scen in scenario_list_run.iterrows():
        # Dapatkan parameter scenario
        filename = scen['scenario']
        load_lvl_iter = scen['load_level']
        f_line_iter = scen['f_line']
        f_loc_iter = scen['f_location']
        f_dur_iter = scen['f_duration']

        iter_start_time = time.perf_counter()

        # Simulasi (implementasikan di sini)
        time.sleep(1)  # Placeholder untuk simulasi


        iter_end_time = time.perf_counter()
        iter_time = iter_end_time - iter_start_time

        # Update DataFrame dan simpan ke file
        scenario_list.loc[scenario_list['scenario'] == filename, 'iter_time'] = iter_time
        scenario_list.to_csv("output/scenario_metadata.csv", index=False)

        # Update progress bar
        pbar.set_postfix({"current_scenario": filename})
        pbar.update(1)


Simulating:   0%|          | 0/8840 [00:01<?, ?iter/s]


KeyboardInterrupt: 

In [ ]:
import os
import itertools
import pandas as pd
from tqdm import tqdm
import time

# Initialize variables
dataset_dir = 'output/dataset'

# Create directories if they don't exist
# dataset_subdirs = ['magnitude', 'angle', 'time', 'out_of_step']
# for dir in dataset_subdirs:
#     os.makedirs(f'{dataset_dir}/{dir}', exist_ok=True)

# Prepare an empty list to store iteration data
iteration_data = []

# Pre-compute total iterations
total_iterations = len(load_level) * len(f_line) * len(f_location) * len(f_duration)
num_digits = len(str(total_iterations))

# Start progress bar
with tqdm(total=total_iterations, desc="Simulating", unit="iter") as pbar:
    iteration_counter = 1
    for load_lvl_iter in load_level:
        # Perform load setup and OPF analysis
        sim.loadSetup(load_lvl_iter, load_default)
        try:
            sim.opfAnalysis('los')
        except AttributeError:
            sim.opfAnalysis('dev')
        
        # Perform short circuit setup and RMS simulation
        for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
            # Start iteration timing
            iter_start_time = time.perf_counter()
            filename = f'scenario_{iteration_counter:0{num_digits}d}'

            # Jalakankan simulasi RMS
            # sim.scSetup(f_loc_iter, f_line_iter, f_dur_iter)
            # sim.rmsSimulation(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)

            # Record the iteration time
            iter_end_time = time.perf_counter()
            iter_time = iter_end_time - iter_start_time

            # Save iteration data to the list
            iteration_data.append({
                "scenario": filename,
                "load_level": load_lvl_iter,
                "f_line": f_line_iter.loc_name,
                "f_location": f_loc_iter,
                "f_duration": f_dur_iter,
                "iter_time": iter_time
            })

            # Update tqdm progress bar
            pbar.set_postfix({
                "current_scenario": filename,
                "completed": f"{pbar.n+1}/{total_iterations}"
            })
            pbar.update(1)
            iteration_counter += 1
            

# After the loop finishes, create a DataFrame from the iteration data
df = pd.DataFrame(iteration_data)

# Optionally, save the DataFrame to a CSV file
df.to_csv('scenario_metadata.csv', index=False)

print("Simulation completed.")


In [ ]:
# sim.app.ResLoadData(res)

# # read the results and time steps and store them as lists
# time = []
# var_values = []
# # load results from file
# sim.app.ResLoadData(res)
# # get number of rows (time steps) in the results file
# n_rows = sim.app.ResGetValueCount(res, 0)

# for i in range(n_rows):
#     time.append(sim.app.ResGetData(res, i, -1)[1])

In [ ]:
res = sim.app.GetFromStudyCase('All calculations.ElmRes')
sim.app.ResLoadData(res)
res.GetVariable(-1)

In [ ]:
def getResult(self, f_loc, f_line, f_dur, load_lvl, elm_name, var_name, var_folder, dir_address):
    self.filename='%s %.0f percent %.2f s %.2f load'%(f_line.loc_name, f_loc, f_dur, load_lvl)
    elm = self.app.GetCalcRelevantObjects(elm_name)
    var = var_name
    res = self.app.GetFromStudyCase('*.ElmRes')
    for i_var in range(len(var)):
        self.comRes = self.app.GetFromStudyCase("ComRes")
        self.comRes.pResult = res
        self.comRes.iopt_exp = 6 # to export as csv
        self.comRes.f_name = dir_address+"\\"+"%s\\%s.csv"%(var_folder[i_var],self.filename) # File Name
        self.comRes.iopt_sep = 0 # to use not the system seperator
        self.comRes.col_Sep = r','
        self.comRes.dec_Sep = r'.'
        self.comRes.iopt_honly = 0 # to export data and not only the header
        self.comRes.iopt_csel = 1 # export only selected variables
        self.comRes.iopt_locn = 1
        self.comRes.ciopt_head = 0
        self.comRes.element = elm
        self.comRes.iopt_tsel = 1
        self.comRes.to = 30
        self.comRes.cfrom = 0
        variable = []
        for i_elm in range(len(elm)):
            variable.append(var[i_var])
        self.comRes.variable = variable
        self.comRes.Execute()

sim.getResult(
    f_loc_iter,
    f_line_iter,
    f_dur_iter,
    load_lvl_iter,
    elm_name = '*.ElmSym',
    var_name = [MONITORED_VARIABLES['*.ElmSym'][0]],
    var_folder = [VAR_FOLDER['*.ElmSym'][0]],
    dir_address = join(parentAddress,"csv"))

In [ ]:
res.GetVariable(-1)

In [ ]:
n_rows = res.GetNumberOfRows()
n_cols = res.GetNumberOfColumns()
res_var = [res.GetVariable(i) for i in range(-1, n_cols)]
res_var_desc = [res.GetDescription(i) for i in range(-1, n_cols)]
res_obj = [res.GetObject(i).loc_name for i in range(-1, n_cols)]
matrix_res = pd.DataFrame({'element': res_obj,
                           'variable': res_var,
                           'variable_description': res_var_desc
                           })
matrix_res
dict_res = {
    str(col + 1): [res.GetValue(row, col)[1] for row in range(n_rows)]
    for col in range(-1, n_cols)
}
pd.DataFrame(dict_res)
# res.GetVariable(4)
# for col in range(-1, n_cols):

# for i in range(n_rows):
#     print(res.GetValue(i, -1)[1])

In [ ]:
iter_time

In [ ]:
for load_lvl_iter in load_level:
    filename_load = 'Kondisi %d persen beban'%(int(round(load_lvl_iter*100,0)))
    if exists(join(parentAddress,'csv',filename_load+'.xlsx')):
        print(filename_load+' exist')
    else:
        print(filename_load+' does not exist')
        sim.loadSetup(load_lvl_iter, load_ori)
        try:
            sim.opfAnalysis()
            sim.ldfAnalysis()
        except AttributeError:
            sim.ldfAnalysis()
        sim.saveLoadandGen(load_lvl_iter, join(parentAddress,'csv'))
    for f_line_iter, f_loc_iter, f_dur_iter in itertools.product(f_line, f_location, f_duration):
        time_start=perf_counter()
        k += 1
        filename = '%s %.0f percent %.2f s %.2f load'%(f_line_iter.loc_name, f_loc_iter, f_dur_iter, load_lvl_iter)
        if exists(join(parentAddress,'csv','rotor_angle',filename+'.csv')):
            print(filename+' exist')
        else:
            print(filename+' does not exist')
            
            sim.shortCircuitSetup(f_loc_iter, f_line_iter, f_dur_iter)
            sim.simulationRMS(MONITORED_VARIABLES, t_start=-100, t_step=10, t_stop=30)
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][0]],
                var_folder = [VAR_FOLDER['*.ElmSym'][0]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmSym',
                var_name = [MONITORED_VARIABLES['*.ElmSym'][1]],
                var_folder = [VAR_FOLDER['*.ElmSym'][1]],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmRes',
                var_name = ['b:tnow'],
                var_folder = ['time'],
                dir_address = join(parentAddress,"csv"))
            
            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][0]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][0]],
                dir_address = join(parentAddress,"csv"))

            sim.getResult(
                f_loc_iter,
                f_line_iter,
                f_dur_iter,
                load_lvl_iter,
                elm_name = '*.ElmTerm',
                var_name = [MONITORED_VARIABLES['*.ElmTerm'][1]],
                var_folder = [VAR_FOLDER['*.ElmTerm'][1]],
                dir_address = join(parentAddress,"csv"))
            
            print("%d / %d - %s %d percent %.2f s %.2f load"%(k,len(f_line)*len(f_location)*len(f_duration)*len(load_level),f_line_iter.loc_name,f_loc_iter,f_dur_iter,load_lvl_iter))
            time_end=perf_counter()
            minute,second=divmod((time_end-time_start),60)
            elapsed_hour, elapsed_sec = divmod((time_end-tstarttot), 3600)
            elapsed_min, elapsed_sec = divmod(elapsed_sec, 60)
            print("Progress %.2f persen - Waktu running per iterasi: %.2f detik - Waktu terlewati: %d jam %d menit %.2f detik"%(100*k/(len(f_line)*len(f_location)*len(f_duration)*len(load_level)),second,elapsed_hour,elapsed_min,elapsed_sec))
hour,second=divmod((time_end-tstarttot),3600)
minute,second=divmod(second,60)
print("Total waktu: %d jam %d menit %.2f detik"%(hour,minute,second))
sim.loadSetup(1, load_ori)